<a href="https://colab.research.google.com/github/seoyeon0905/mimic-readmission-analysis/blob/main/notebooks/07_lessons_from_readmission_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <지금까지의 흐름>
- 01: 데이터 구조 이해(테이블 관계 파악)
- 02: 코호트 + 라벨 정의(연구 설계)
- 03: EDA(단일 변수 수준에서 "뭔가 차이는 있는 듯")
- 04: Baseline 모델 -> 이 정도 정보로는 예측 안 된다
- 05: Risk factor 해석 -> 그래도 이런 패턴은 있다
- 06: Feature 확장 + 모델 비교 -> 그럼 더 넣어봤는데 여전히 쉽진 않다

# 이 노트북의 목적
본 노트북은 ICU 환자의 30일 이내 재입원 분석 프로젝트를 수행하며 얻은 주요 교훈과 한계를 정리하기 위한 것이다.
이 노트북의 목적은 예측 성능을 추가로 개선하는 것이 아니라 왜 성능이 제한적이었는지, 그리고 이러한 결과가 실제 병원 환경에서 어떤 의미를 가지는지를 데이터 분석 관점에서 정리하는 데 있다.

# 본 프로젝트에서 시도한 것들
본 프로젝트에서는 ICU 환자의 30일 이내 재입원 여부를 분석하기 위해 다음과 같은 단계적 분석 파이프라인을 수행하였다.

- 환자별 첫 ICU 입원을 기준으로 cohort 정의
- 입·퇴원 시점을 활용한 30일 재입원 라벨 생성
- 재입원군과 비재입원군 간의 기초 EDA
- 제한된 변수(age, ICU LOS)를 활용한 baseline 모델 구축
- 입원 유형, 입원 경로, 퇴원 경로, 보험 유형을 포함한 feature 확장
- 선형 모델(Logistic Regression)과 비선형 모델(Random Forest) 비교

이를 통해 단순한 변수 구성에서부터
입원 맥락 정보를 포함한 확장된 모델까지
점진적으로 분석 범위를 넓혀가며 재입원 예측 가능성을 탐색하였다.

# 핵심 결과 요약
모든 모델에서 전반적인 예측 성능은 제한적인 수준에 머물렀다.

- Baseline Logistic Regression
  - AUROC ≈ 0.48
  - Recall = 0
  - PR-AUC ≈ 0.09
- Extended Logistic Regression
  - AUROC 감소
  - Recall 개선 없음
- Random Forest (확장 feature)
  - AUROC ≈ 0.50
  - Recall = 0
  - PR-AUC ≈ 0.28로 상승

즉, 입원 관련 변수를 추가하고 비선형 모델을 적용하였음에도
기본 threshold 기준에서는 재입원 환자를 안정적으로 탐지하지 못하였다.

# ICU 재입원 예측이 어려운 이유
1. 심각한 클래스 불균형
- 본 cohort에서 30일 이내 재입원 환자는 전체의 약 8%에 불과하였다.
이는 전형적인 rare event 문제로
accuracy와 같은 지표가 쉽게 착시를 일으킬 수 있는 환경이다.

2. 표본 수의 한계
- MIMIC-III demo 데이터셋은 총 100명의 환자만을 포함하고 있으며
양성 사례는 8명에 불과하다.
이러한 환경에서는 모델이 안정적인 decision boundary를 학습하기 어렵다.

3. 재입원이라는 outcome의 특성
- 재입원은 입원 당시의 임상적 중증도뿐만 아니라
퇴원 이후 관리, 외래 추적, 사회·경제적 요인,
그리고 데이터에 기록되지 않는 우연적 사건의 영향을 함께 받는다.
- 이러한 요소들은 구조화된 병원 데이터에 충분히 반영되지 않는 경우가 많다.


# 방법론적 관점에서의 핵심 교훈
1. 위험 요인과 예측 변수는 다르다.
- ICU 재원 기간과 같은 변수는 재입원군과 비재입원군 간의
해석 가능한 차이를 보였으나
이 차이가 곧바로 높은 예측 성능으로 이어지지는 않았다.
- 이는 위험 요인 분석과 예측 모델링이
서로 다른 목적을 가진다는 점을 보여준다.

2. Feature 확장만으로는 한계가 있다.
- 입원 관련 변수를 추가함으로써
Random Forest 모델의 PR-AUC는 일부 개선되었으나
recall 기준의 성능 개선은 관찰되지 않았다.

이는 모델의 복잡도를 높이는 것만으로는
데이터 자체의 정보 한계를 극복하기 어렵다는 점을 시사한다.

3. 성능이 나오지 않는 분석도 의미가 있다.
- 예측 성능이 제한적이라는 결과 자체가
데이터의 한계와 문제 구조를 이해하는 데 중요한 정보가 된다.

실제 의료 데이터 분석에서는
모델이 잘 작동하지 않는 이유를 설명하는 능력 또한 중요한 역량이다.


# 병원 실무 관점에서의 시사점
실제 병원 환경에서 재입원 분석은
개별 환자의 재입원을 정확히 예측하는 도구라기보다는

- 재입원 위험이 상대적으로 높은 환자군을 파악하고
- 퇴원 이후 모니터링 대상 집단을 정의하며
- 병원 내 치료·퇴원 프로세스를 점검하기 위한 참고 자료

로 활용되는 경우가 많다.

따라서 이러한 모델은 단독 의사결정 도구가 아니라
임상적 판단을 보조하는 분석 결과로 해석되어야 한다.


# 분석의 한계
본 분석은 다음과 같은 한계를 가진다.

- 소규모 demo 데이터 사용
- 진단 코드, 처치 코드 등 세부 임상 변수 부재
- 외부 데이터에 대한 검증 미수행
- 인과 추론을 목적으로 하지 않음

이러한 한계는 본 프로젝트가
교육 및 포트폴리오 목적의 분석임을 고려하여
의도적으로 수용한 것이다.


# 다음 프로젝트를 위한 방향성
본 프로젝트를 통해 얻은 교훈을 바탕으로
취업용 포트폴리오 프로젝트에서는

- 더 큰 규모의 공공 의료 데이터를 사용하고
- 예측 성능보다는 기술 통계와 비교 분석에 집중하며
- 병원이나 공공기관이 바로 활용할 수 있는
명확한 결과를 도출하는 방향이 더 적합하다고 판단하였다.

# 최종 요약
```markdown
본 프로젝트는 ICU 재입원 예측의 전 과정을 경험하며
의료 데이터 분석에서 모델 성능뿐만 아니라
문제 구조와 데이터 한계를 이해하는 것이
얼마나 중요한지를 보여준다.

의료 현장에서는 이러한 한계를 인식하고 설명할 수 있는 분석가의 역할이
중요하다는 점을 본 분석을 통해 확인하였다.
```